In [1]:
from datasets import load_dataset
from datasets import ClassLabel

SexWeibo = load_dataset("csv", data_files="SexWeibo.csv")['train']
SexWeibo_dataset = SexWeibo.remove_columns(['weibo_id','user_gender','user_location','user_following','user_follower','weibo_like','weibo_repost', 'weibo_comment', 'weibo_date'])
SexWeibo_dataset = SexWeibo_dataset.rename_column('weibo_text','text_cn')
# SexWeibo_dataset = SexWeibo_dataset.filter(lambda x: x['keyword'] is not None and x['keyword'].find('厌女') != -1)

sexism_column = ['not sexist'] * len(SexWeibo_dataset)
SexWeibo_dataset = SexWeibo_dataset.add_column('label_sexist', sexism_column)

def identify_sexist(row):
    if row['keyword'] is not None and row['keyword'].find('厌女') != -1:
        row['label_sexist'] = 'sexist'
    return row

SexWeibo_dataset = SexWeibo_dataset.map(identify_sexist, batched=False)

dataset = SexWeibo_dataset.class_encode_column("label_sexist")

# ## remove long rows with long text 
dataset = dataset.filter(lambda x: len(x['text_cn'])<200)


In [2]:
dataset

Dataset({
    features: ['text_cn', 'keyword', 'label_sexist'],
    num_rows: 720
})

In [3]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline(model="Helsinki-NLP/opus-mt-zh-en", device=device)
# pipe = pipeline(model="Helsinki-NLP/opus-mt-zh-en")


def trans_CN_to_EN(cn: str) -> str:
    return pipe(cn)[0]['translation_text']

def add_text(row):
    row['text'] = trans_CN_to_EN(row['text_cn'])
    return row


In [5]:
## Translate the dataset
new_dataset = dataset.map(add_text)


## Split the dataset

# new_features = dataset.features.copy()
# new_features['label_sexist'] = ClassLabel(names=["not sexist", "sexist"])
# new_dataset = dataset.cast(new_features)

split_dataset = new_dataset.train_test_split(0.1,stratify_by_column='label_sexist')
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']
split_again = train_dataset.train_test_split(0.1,stratify_by_column='label_sexist')
train_dataset = split_again['train']
validation_dataset = split_again['test']

## combine the dataset

from datasets import DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})
print(dataset_dict)


## Upload the dataset

import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/SWSR-SexWeibo")

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

/home/yzheng/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import json

file_path = "credentials.json"

with open(file_path, 'r') as json_file:
    token_data = json.load(json_file)

token = token_data.get("huggingface_token")
if token:
    print(f"Token read successfully")
else:
    print("Error: Token not found in the JSON file.")
    
import huggingface_hub

huggingface_hub.login(token)

dataset_dict.push_to_hub("yangezheng/CMSB")